# Feature Extraction

### Introduction
This document outlines the feature extraction process conducted on CT-scan data of lung patients. The objective of this feature extraction is to analyze and extract relevant information from medical images. The project encompasses the utilization of two distinct Python libraries, namely 'pylidc' and 'pyradiomics,' to enable comprehensive feature extraction.

### Libraries and Dependencies

We start by importing relevant libraries.

In [2]:
import pylidc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

### PyLIDC Feature Extraction

#### Data Structure Initialization

Our first step involves creating a Pandas DataFrame to hold the information to be extracted:

In [3]:
df_pylidc = pd.DataFrame(columns= [
                            'patient_id', 
                            'annotation_id',
                            'scan_id',
                            
                            'slice_thickness',
                            'pixel_spacing',

                            'subtlety', 
                            'internalStructure', 
                            'calcification', 
                            'sphericity', 
                            'margin', 
                            'lobulation', 
                            'spiculation', 
                            'texture', 
                        
                            'diameter',
                            'surface_area',
                            'volume',

                            'malignancy',
                            ])

#### Data Extraction and Storage

Using the 'pylidc' library, the relevant data is extracted and populated into the DataFrame. The data is subsequently saved in a CSV file for further analysis.

In [4]:
ann = pylidc.query(pylidc.Annotation).all()

for i in range(len(ann)):
    att  = dict((col, "") for col in df_pylidc.columns)

    # patient and annotation identification
    att['patient_id'] = ann[i].scan.patient_id
    att['annotation_id'] = ann[i].id  
    att['scan_id'] = ann[i].scan.id

    # features
    st = pylidc.query(pylidc.Scan.slice_thickness).filter(pylidc.Scan.id == att['scan_id'])
    s = str(st[0])
    att['slice_thickness'] = float(s[1:4])
    ps = pylidc.query(pylidc.Scan.pixel_spacing).filter(pylidc.Scan.id == att['scan_id'])
    p = str(ps[0])
    att['pixel_spacing'] = float(p[1:5])

    att['subtlety'] = ann[i].subtlety
    att['internalStructure'] = ann[i].internalStructure 
    att['calcification'] = ann[i].calcification 
    att['sphericity'] = ann[i].sphericity
    att['margin'] = ann[i].margin  
    att['lobulation'] = ann[i].lobulation
    att['spiculation'] = ann[i].spiculation 
    att['texture'] = ann[i].texture

    att['diameter'] = ann[i].diameter
    att['surface_area'] = ann[i].surface_area
    att['volume'] = ann[i].volume

    # target
    att['malignancy'] = ann[i].malignancy  

    df_pylidc = df_pylidc.append(att, ignore_index=True)

df_pylidc.to_csv('pylidc_features.csv', sep=',', index=False)

C:\Users\anton\AppData\Local\Temp\ipykernel_24752\2568502329.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_pylidc = df_pylidc.append(att, ignore_index=True)
C:\Users\anton\AppData\Local\Temp\ipykernel_24752\2568502329.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_pylidc = df_pylidc.append(att, ignore_index=True)
C:\Users\anton\AppData\Local\Temp\ipykernel_24752\2568502329.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_pylidc = df_pylidc.append(att, ignore_index=True)
C:\Users\anton\AppData\Local\Temp\ipykernel_24752\2568502329.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_pylidc = df_pylidc.append(

#### Data Exploration

Following the feature extraction process, an initial exploration of the data is conducted. Key aspects include:

- Determining the total number of individual patients assessed in the dataset.
- Providing a summary of the DataFrame's structure.

In [ ]:
df_pylidc = pd.read_csv('pylidc_features.csv', index_col=False)
df_pylidc.head()

,patient_id,annotation_id,scan_id,slice_thickness,pixel_spacing,subtlety,internalStructure,calcification,sphericity,margin,lobulation,spiculation,texture,diameter,surface_area,volume,malignancy
0,LIDC-IDRI-0078,1,1,3.0,0.65,5,1,6,3,4,1,1,5,20.840585,1124.125177,2439.30375,3
1,LIDC-IDRI-0078,2,1,3.0,0.65,4,1,6,4,4,1,2,5,19.500000,1135.239277,2621.82375,3
2,LIDC-IDRI-0078,3,1,3.0,0.65,5,1,4,3,5,2,3,5,23.300483,1650.898027,4332.31500,4
3,LIDC-IDRI-0078,4,1,3.0,0.65,5,1,6,4,2,4,1,5,32.810517,1994.684094,5230.33875,5
4,LIDC-IDRI-0078,5,1,3.0,0.65,4,1,6,4,2,3,1,4,20.891206,1130.172711,2443.74000,4


In [ ]:
patients = np.unique(df_pylidc.patient_id)
print(f"Total number of patients: {len(patients)} \n")

df_pylidc.info()

Total number of patients: 875 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6859 entries, 0 to 6858
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   patient_id         6859 non-null   object 
 1   annotation_id      6859 non-null   int64  
 2   scan_id            6859 non-null   int64  
 3   slice_thickness    6859 non-null   float64
 4   pixel_spacing      6859 non-null   float64
 5   subtlety           6859 non-null   int64  
 6   internalStructure  6859 non-null   int64  
 7   calcification      6859 non-null   int64  
 8   sphericity         6859 non-null   int64  
 9   margin             6859 non-null   int64  
 10  lobulation         6859 non-null   int64  
 11  spiculation        6859 non-null   int64  
 12  texture            6859 non-null   int64  
 13  diameter           6859 non-null   float64
 14  surface_area       6859 non-null   float64
 15  volume             6859 non-null   float

#### Standardizing Indexes

To enhance data consistency, the DataFrame is sorted by 'patient_id' and 'annotation_id,' and a new index column, 'Ann_id,' is introduced to represent the order of annotations for each patient.

In [ ]:
df_pylidc.sort_values(by=['patient_id', 'annotation_id'], inplace=True)

df_pylidc['Ann_id'] = df_pylidc.groupby('patient_id').cumcount() + 1
df_pylidc = df_pylidc[['Ann_id'] + [col for col in df_pylidc.columns if col != 'Ann_id']]

Aditionally, redundant columns are removed, and the modified DataFrame is saved to a new CSV file.

In [ ]:
for i in range(len(df_pylidc)):
    df_pylidc.at[i,'Id'] = df_pylidc.at[i,'patient_id'] + '-' + str(df_pylidc.at[i,'Ann_id'])

df_pylidc = df_pylidc[['Id'] + [col for col in df_pylidc.columns if col != 'Id']]
df_pylidc = df_pylidc.drop(columns=['Ann_id', 'annotation_id', 'scan_id'])

df_pylidc.to_csv('pylidc_features_fixed.csv', sep=',', index=False)

### PyRadiomics Feature Extraction

'pyradiomics' does not directly process DICOM files. In their documentation, though, it is possible to find a experimental script (pyradiomics-dcm.py) designed to extract features from DICOM data by incorporating 'plastimatch' and 'dcmqi'.

We modified this script to append new lines to an already existing, but containing only the headers, CSV file and used it for our feature extraction.

This script was given as input:
- a directory with the input DICOM series
- the file name pointing to a DICOM Segmentation Image (DICOM SEG) object
- the file name specifying the extraction parameters in a YAML format
- the file name referring to a TSV dictionary mapping 'pyradiomics' feature names to IBSI-defined features.
- the ID for that segmentation

#### Procedure

The feature extraction procedure involves iterating through all patient data and respective segmentations to process feature extraction. For each patient:

1. The script navigates to the patient's folder, ignoring folders dedicated to storing output and other miscellaneous files.  
2. The CT-scan folder, which contains individual segmentations and the input DICOM series, is identified.  
3. The script extracts features for each segmentation, naming each line as 'Patient_Name-Segmentation_Number'.  

In [ ]:
main_directory = os.listdir()

for patient in main_directory:
    # enter each patients folder, ignoring the folders for storing output and other files that may be inside the main directory
    if (not os.path.isdir(patient)) or patient == "OutputSR" or patient == "TempDir":
        continue
    
    folders = os.listdir(patient) # list of folders inside patient (CT-scan and X-ray)
    path = ""
    content = 0
    
    # finding the CT-scan folder (has the biggest content - directories with each segmentation and directory with the input DICOM series)
    for folder in folders:
        segmentations_or_series = os.listdir(patient + "\\" + folder)
        if (len(segmentations_or_series) > content):
            content = len(segmentations_or_series)
            path = folder
    
    folders = os.listdir(patient + "\\" + path)
    main = ""

    # finding the series folder
    for folder in folders:
        if not "Annotation" in folder:
            main = folder
            break
        
    seg_index = 1
    
    # extract features for each segmentation and naming each line with Patient_Name-Segmentation_Number
    for folder in folders:
        if "Segmentation" in folder:
            print("\n\nPACIENTE NUMERO " + patient + " - SEGMENTAÇÃO " + str(seg_index))
            os.system(f'cmd /c """python pyradiomics-dcm.py --input-image-dir "{patient}\{path}\{main}" --input-seg-file "{patient}\{path}\{folder}\\1-1.dcm" --output-dir OutputSR --temp-dir TempDir --parameters Pyradiomics_Params.yaml --features-dict featuresDict.tsv --name {patient}-{seg_index}"""')
            seg_index+=1


The previous script takes days to run for all patients, which is why we found it unwise to run it again inside the notebook.

#### Data Anomalies

During feature extraction using 'pyradiomics,' certain issues arise:

- Some images are too small to apply the Log filter.
- Some segmentations lack the necessary dimensions or voxels for processing.

#### Data Exploration

Following the 'pyradiomics' feature extraction, a comprehensive examination of the dataset is performed. Key insights include:

- Determining the total number of annotations assessed in the 'pyradiomics' dataset.
- Analyzing the structure of the DataFrame.

In [6]:
df_pyradiomics = pd.read_csv('pyradiomics_features.csv', index_col=False)

annotations = np.unique(df_pyradiomics.id)
print(f"Total number of annotations: {len(annotations)} \n")

df_pyradiomics.info()

Total number of patients: 4688 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4696 entries, 0 to 4695
Columns: 1599 entries, id to logarithm_ngtdm_Strength
dtypes: float64(1565), int64(4), object(30)
memory usage: 57.3+ MB


### Data Correction

When looking at the whole dataframe, we were able to see that the lines where the Log Filter wasn't correctly applied were misplaced.  
To address this, the affected samples are shifted to correct positions. The adjusted data is saved to a new CSV file.

In [7]:
df_pyradiomics.iloc[:, 143:] = df_pyradiomics.iloc[:, 143:].apply(lambda x: x.shift(273) if x.isna().any() else x, axis=1)
df_pyradiomics.to_csv('pyradiomics_extraction_fixed.csv', index=False)

### Bibliographic References

#### PyLIDC

Documentation: https://pylidc.github.io 

#### PyRadiomics

Documentation: https://pyradiomics.readthedocs.io/en/latest/

Publication: Van Griethuysen, J. J. M., Fedorov, A., Parmar, C., Hosny, A., Aucoin, N., Narayan, V., Beets-Tan, R. G. H., Fillon-Robin, J. C., Pieper, S., Aerts, H. J. W. L. (2017). 
Computational Radiomics System to Decode the Radiographic Phenotype. Cancer Research, 77(21), e104–e107. at https://doi.org/10.1158/0008-5472.CAN-17-0339.
